In [27]:
import os
import json 
import pandas as pd
import traceback

In [28]:
import langchain_community

In [29]:
from langchain.chat_models import ChatOpenAI

In [30]:
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
KEY= os.getenv("OPENAI_API_KEY")

In [32]:
llm= ChatOpenAI(openai_api_key= KEY, model_name= "gpt-3.5-turbo", temperature= 0.5)

In [33]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001A5419F17C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001A5419EFF40>, temperature=0.5, openai_api_key='sk-proj-Yn2iwjKb9u2BGje9WiLqT3BlbkFJsPwxtLmiHo69qCpfCBbP', openai_proxy='')

In [34]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [35]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [36]:
TEMPELATE= """"
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \ 
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure that the questions are not repeated and check all the questions to be confirming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
 
 """

In [37]:
quiz_generation_prompt= PromptTemplate(
    input_variables= ["text", "number", "subject", "tone", "response_json"],
    template= TEMPELATE
)

In [38]:
quiz_chain= LLMChain(llm= llm, prompt= quiz_generation_prompt, output_key= "quiz", verbose= True)

In [39]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [40]:
quiz_evaluaion_prompt= PromptTemplate(input_variables=["subject", "quiz"], template= TEMPELATE)

In [41]:
review_chain= LLMChain(llm= llm, prompt= quiz_evaluaion_prompt, output_key= "review", verbose= True)

In [42]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [43]:
file_path= r"C:\Users\ayush\last\data.txt"

In [44]:
file_path

'C:\\Users\\ayush\\last\\data.txt'

In [45]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [46]:
print(TEXT)

The Heroes of Olympus is a pentalogy of fantasy-adventure novels written by American author Rick Riordan. The novels detail a conflict between Greek demigods, Roman demigods, and Gaea. In the fourth book of the series, there is also a fight against Tartarus, which, in Greek mythology, was the darkest and deepest point of the Underworld.

The series can be read as a standalone volume, but is meant to be read after Percy Jackson & the Olympians. Riordan introduces Roman mythology in the series alongside several new characters, primarily from the Roman Camp Jupiter. The first book of the series, The Lost Hero, was published on October 12, 2010.[4] The final entry in the series, The Blood of Olympus, was published on October 7, 2014.[5] The story is continued in the sequel series The Trials of Apollo.
The Heroes of Olympus is centered around a prophecy, introduced in The Last Olympian, that predicted seven demigods would unite to protect the world from an awakening new enemy, the Earth god

In [47]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [48]:
NUMBER=5 
SUBJECT="fiction"
TONE="simple"

In [49]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
"
Text:The Heroes of Olympus is a pentalogy of fantasy-adventure novels written by American author Rick Riordan. The novels detail a conflict between Greek demigods, Roman demigods, and Gaea. In the fourth book of the series, there is also a fight against Tartarus, which, in Greek mythology, was the darkest and deepest point of the Underworld.

The series can be read as a standalone volume, but is meant to be read after Percy Jackson & the Olympians. Riordan introduces Roman mythology in the series alongside several new characters, primarily from the Roman Camp Jupiter. The first book of the series, The Lost Hero, was published on October 12, 2010.[4] The final entry in the series, The Blood of Olympus, was published on October 7, 2014.[5] The story is continued in the sequel series The Trials of Apollo.
The Heroes of Olympus is centered around a prophecy, introduced in The Last Olympi

In [50]:
response

{'text': 'The Heroes of Olympus is a pentalogy of fantasy-adventure novels written by American author Rick Riordan. The novels detail a conflict between Greek demigods, Roman demigods, and Gaea. In the fourth book of the series, there is also a fight against Tartarus, which, in Greek mythology, was the darkest and deepest point of the Underworld.\n\nThe series can be read as a standalone volume, but is meant to be read after Percy Jackson & the Olympians. Riordan introduces Roman mythology in the series alongside several new characters, primarily from the Roman Camp Jupiter. The first book of the series, The Lost Hero, was published on October 12, 2010.[4] The final entry in the series, The Blood of Olympus, was published on October 7, 2014.[5] The story is continued in the sequel series The Trials of Apollo.\nThe Heroes of Olympus is centered around a prophecy, introduced in The Last Olympian, that predicted seven demigods would unite to protect the world from an awakening new enemy, 

In [51]:
quiz= response.get("quiz")

In [52]:
quiz=json.loads(quiz)

In [53]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [54]:
quiz_table_data

[{'MCQ': 'Who is the main protagonist of The Heroes of Olympus series?',
  'Choices': 'a: Piper McLean | b: Leo Valdez | c: Annabeth Chase | d: Percy Jackson',
  'Correct': 'd'},
 {'MCQ': 'What is the rare hereditary gift that Piper McLean possesses?',
  'Choices': 'a: Telekinesis | b: Charmspeak | c: Invisibility | d: Shape-shifting',
  'Correct': 'b'},
 {'MCQ': 'Which character can summon and control fire in the series?',
  'Choices': 'a: Nico di Angelo | b: Jason Grace | c: Hazel Levesque | d: Frank Zhang',
  'Correct': 'b'},
 {'MCQ': 'Who is the daughter of the Roman goddess of war, Bellona?',
  'Choices': 'a: Reyna Avila Ramírez-Arellano | b: Thalia Grace | c: Rachel Elizabeth Dare | d: Grover Underwood',
  'Correct': 'a'},
 {'MCQ': 'What is the main enemy that the demigods unite to protect the world from in The Heroes of Olympus series?',
  'Choices': 'a: Hades | b: Zeus | c: Gaea | d: Ares',
  'Correct': 'c'}]

In [57]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,Who is the main protagonist of The Heroes of O...,a: Piper McLean | b: Leo Valdez | c: Annabeth ...,d
1,What is the rare hereditary gift that Piper Mc...,a: Telekinesis | b: Charmspeak | c: Invisibili...,b
2,Which character can summon and control fire in...,a: Nico di Angelo | b: Jason Grace | c: Hazel ...,b
3,Who is the daughter of the Roman goddess of wa...,a: Reyna Avila Ramírez-Arellano | b: Thalia Gr...,a
4,What is the main enemy that the demigods unite...,a: Hades | b: Zeus | c: Gaea | d: Ares,c


In [58]:
quiz= pd.DataFrame(quiz_table_data)

In [59]:
quiz.to_csv("percy.csv",index=False)
